# AdventureWorks Försäljningsanalys
I denna analys undersöker vi  AdventureWorks-databasen för att besvara affärsfrågor som exempelvis: 
- Produktkategorier och deras försäljning
- Försäljningstrender över tid
- Regional försäljning

Vi använder SQL för att hämta data och Python för visualisering och analys.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import numpy as np
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus


server = "localhost"
database = "AdventureWorks2025"
driver = quote_plus("ODBC Driver 18 for SQL Server")


connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver}&trusted_connection=yes&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)


try:
    with engine.connect():
        print("Anslutning till SQL Server lyckades")
except Exception as e:
    print("Kunde inte ansluta", e)



In [ ]:
def query_df(sql: str):
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn)

## Definition av försäljning i denna notebook

I visualiseringar som bygger på **Sales.SalesOrderDetail** används `LineTotal`, vilket motsvarar varuvärde (≈ orderns `SubTotal`, exkl. skatt och frakt).
I visualiseringar som bygger på **Sales.SalesOrderHeader** används `TotalDue`, vilket inkluderar `SubTotal + TaxAmt + Freight`. Därför kan totalsummor skilja sig mellan vissa grafer.



## Visualisering 1: Antal produkter per kategori

**Affärsfråga:** Hur många produkter finns i varje kategori?

**Tabeller:**  
- `Production.ProductCategory`  
- `Production.ProductSubcategory`  
- `Production.Product`

**Metod:**  
- JOIN kategori → subkategori → produkt  
- Räknar **unika** produkter per kategori (`COUNT(DISTINCT ...)`)  
- Visualiserar resultatet som ett **vertikalt stapeldiagram**



In [ ]:
query_vis1 = """
SELECT 
    pc.Name AS CategoryName,
    COUNT(DISTINCT p.ProductID) AS ProductCount
 
 
FROM Production.ProductCategory pc
INNER JOIN Production.ProductSubcategory psc ON pc.ProductCategoryID = psc.ProductCategoryID
INNER JOIN Production.Product p ON psc.ProductSubcategoryID = p.ProductSubcategoryID
GROUP BY pc.Name
ORDER BY ProductCount DESC;
"""

df_vis1 = query_df(query_vis1)


fig, ax = plt.subplots(figsize=(10,6))
bars = ax.bar(df_vis1['CategoryName'], df_vis1['ProductCount'], color='steelblue', alpha=0.8)

for bar in bars:
    height = bar.get_height()
    ax.text(
    bar.get_x() + bar.get_width()/2,
    height,
    f'{int(height)}',
    ha='center',
    va='bottom'
    )

ax.set_xlabel('Produktkategori', fontsize=12)
ax.set_ylabel('Antal produkter', fontsize=12)
ax.set_title('Antal produkter per kategori', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### Insikter – Antal produkter per kategori

- **Flest produkter:** Components – 134 produkter  
- **Minst produkter:** Accessories – 29 produkter  

Detta tyder på att sortimentet är mest omfattande inom komponenter/reservdelar. Accessories kan vara ett område att utveckla om företaget vill bredda erbjudandet eller driva mer merförsäljning.


## Visualisering 2: Försäljning per produktkategori

**Affärsfråga:** Vilka produktkategorier genererar mest intäkter?

**Tabeller:**  
- `Production.ProductCategory`  
- `Production.ProductSubcategory`  
- `Production.Product`  
- `Sales.SalesOrderDetail`

**Metod:**  
- JOIN kategori → subkategori → produkt → orderrad  
- Summerar försäljning per kategori (`SUM(LineTotal)`) 
- Försäljning beräknas som SUM(LineTotal) från Sales.SalesOrderDetail (orderradsbelopp – används för produkt/kategori)
- Sorterar från högst till lägst  
- Visualiserar resultatet som ett **horisontellt stapeldiagram** (högsta överst)


In [ ]:
query_vis2 = """
SELECT
    pc.Name AS CategoryName,
    SUM(sod.LineTotal) AS TotalSales


FROM Production.ProductCategory pc
INNER JOIN Production.ProductSubcategory psc ON pc.ProductCategoryID = psc.ProductCategoryID
INNER JOIN Production.Product p ON psc.ProductSubcategoryID = p.ProductSubcategoryID
INNER JOIN Sales.SalesOrderDetail sod ON p.ProductID = sod.ProductID
GROUP BY pc.Name
ORDER BY TotalSales DESC;
"""

df_vis2 = query_df(query_vis2)
df_vis2


fig, ax = plt.subplots(figsize=(10,6))
bars = ax.barh(df_vis2["CategoryName"], df_vis2["TotalSales"], color="steelblue", alpha=0.8)

ax.invert_yaxis()

ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: f"{x:,.0f}"))

for bar in bars:
    width = bar.get_width()
    ax.text(
        width,
        bar.get_y() + bar.get_height()/2,
        f"{width:,.0f}",
        va="center",
        ha="left"
    )

ax.set_xlabel("Total försäljning (LineTotal)", fontsize=12)
ax.set_ylabel("Produktkategori", fontsize=12)
ax.set_title("Total försäljning per produktkategori", fontsize=14, fontweight="bold")

max_val = df_vis2["TotalSales"].max()
ax.set_xlim(0, max_val * 1.12) 

plt.tight_layout()
plt.show()

### Insikter – Total försäljning per produktkategori

- **Störst kategori:** Bikes – 94,651,172.70  
- **Minst kategori:** Accessories – 1,272,072.88  

Försäljningen domineras tydligt av Bikes, medan Accessories står för en mycket mindre del av intäkterna. Detta kan tyda på att kärnaffären ligger i cyklar och att tillbehör i högre grad fungerar som kompletterande försäljning.



## Visualisering 3: Försäljningstrend per månad

**Affärsfråga:** Hur har försäljningen utvecklats över tid?

**Tabeller:**  
- `Sales.SalesOrderHeader`

**Metod:**  
- Aggregerar försäljning per månad (minst 12 månader)  
- Summerar total försäljning per månad (`SUM(TotalDue)` eller motsvarande)  
- Försäljning i trendanalysen beräknas som SUM(TotalDue) från Sales.SalesOrderHeader (ordertotal inkl. skatt och frakt).
- Drill-down på produktmix beräknas med SUM(LineTotal) från Sales.SalesOrderDetail eftersom det går att bryta ner per produkt/kategori.
- Sorterar kronologiskt (äldst → nyast)  
- Visualiserar resultatet som ett **linjediagram** med tydlig tidsaxel



In [ ]:
query_vis3 = """
SELECT
    DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1) AS MonthStart,
    COUNT(*) AS OrderCount,
    SUM(TotalDue) AS TotalSales,
    AVG(TotalDue) AS AvgOrderValue
FROM Sales.SalesOrderHeader
GROUP BY DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1)
ORDER BY MonthStart;
"""


df_vis3 = query_df(query_vis3)
df_vis3["MonthStart"] = pd.to_datetime(df_vis3["MonthStart"])
df_vis3


fig, ax = plt.subplots(figsize=(10,6))

ax.plot(df_vis3["MonthStart"], df_vis3["TotalSales"])

ax.set_title("Försäljningstrend per månad", fontsize=14, fontweight="bold")
ax.set_xlabel("Månad", fontsize=12)
ax.set_ylabel("Total försäljning (TotalDue)", fontsize=12)

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))   # var 3:e månad (snyggt för 38 månader)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.xticks(rotation=45)

ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: f"{x:,.0f}"))

plt.tight_layout()
plt.show()


In [ ]:
max_row = df_vis3.loc[df_vis3["TotalSales"].idxmax()]
min_row = df_vis3.loc[df_vis3["TotalSales"].idxmin()]

print("Högsta månaden:", max_row["MonthStart"].strftime("%Y-%m"), f"({max_row['TotalSales']:,.2f})")
print("Lägsta månaden:", min_row["MonthStart"].strftime("%Y-%m"), f"({min_row['TotalSales']:,.2f})")

In [ ]:
query_vis3_june_mix = """
SELECT pc.Name AS CategoryName, SUM(sod.LineTotal) AS Sales
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.SalesOrderDetail sod ON soh.SalesOrderID = sod.SalesOrderID
INNER JOIN Production.Product p ON sod.ProductID = p.ProductID
INNER JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
INNER JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
WHERE soh.OrderDate >= '2025-06-01' AND soh.OrderDate < '2025-07-01'
GROUP BY pc.Name
ORDER BY Sales DESC;
"""
df_vis3_june_mix = query_df(query_vis3_june_mix)
df_vis3_june_mix

In [ ]:
query_vis3_may_june_mix = """
SELECT
    DATEFROMPARTS(YEAR(soh.OrderDate), MONTH(soh.OrderDate), 1) AS MonthStart,
    pc.Name AS CategoryName,
    SUM(sod.LineTotal) AS Sales
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.SalesOrderDetail sod ON soh.SalesOrderID = sod.SalesOrderID
INNER JOIN Production.Product p ON sod.ProductID = p.ProductID
INNER JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
INNER JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
WHERE soh.OrderDate >= '2025-05-01' AND soh.OrderDate < '2025-07-01'
GROUP BY DATEFROMPARTS(YEAR(soh.OrderDate), MONTH(soh.OrderDate), 1), pc.Name
ORDER BY MonthStart, Sales DESC;
"""

df_vis3_may_june_mix = query_df(query_vis3_may_june_mix)
df_vis3_may_june_mix

In [ ]:
query_vis3_seasonality = """
SELECT
    MONTH(OrderDate) AS MonthNo,
    DATENAME(MONTH, DATEFROMPARTS(2025, MONTH(OrderDate), 1)) AS MonthName,
    AVG(CAST(TotalDue AS float)) AS AvgOrderValue,
    SUM(TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader
GROUP BY MONTH(OrderDate), DATENAME(MONTH, DATEFROMPARTS(2025, MONTH(OrderDate), 1))
ORDER BY MonthNo;
"""

df_vis3_seasonality = query_df(query_vis3_seasonality)
df_vis3_seasonality

### Insikter – Försäljningstrend per månad

- **Övergripande trend:** Försäljningen varierar över tid men ligger generellt på en högre nivå **2024–2025** jämfört med **2022–2023**, vilket tyder på en övergripande ökning.
- **Högsta månaden:** **2025-04** – **5,847,164.69** i total försäljning  
- **Lägsta månaden:** **2025-06** – **52,478.19**

**Avvikelse (juni 2025):**  
Juni 2025 sticker ut med extremt låg försäljning trots att data finns nästan hela månaden (t.o.m. **2025-06-29**). Kontroller visar att försäljningen under juni 2025 nästan enbart kommer från:
- **Accessories:** 31,591.32  
- **Clothing:** 15,900.23  
och att **Bikes saknas** i juni. Detta gör att genomsnittligt ordervärde blir mycket lågt (**57.99**), vilket drar ner totalförsäljningen kraftigt.

**Säsongsmönster:**  
I genomsnitt är **juni** den starkaste månaden (**12,614,619.62**) och **november** den svagaste (**7,372,049.02**). Även **juli** och **september** ligger högt, vilket tyder på högre försäljning under sommaren och tidig höst. Att juni vanligtvis är stark gör att **juni 2025** blir en extra tydlig avvikelse som kan vara värd att undersöka vidare (t.ex. datatäckning, produktmix eller specifika affärshändelser).



## Visualisering 4: Försäljning och antal ordrar per år

**Affärsfråga:** Hur ser total försäljning och antal ordrar ut per år?

**Tabeller:**  
- `Sales.SalesOrderHeader`

**Metod:**  
- Grupperar per år  
- Beräknar både **total försäljning** och **antal ordrar**  
- Sorterar kronologiskt (äldst → nyast)  
- Visualiserar resultatet som ett **grupperat stapeldiagram** (två mått per år)
- Försäljning beräknas som SUM(TotalDue) från Sales.SalesOrderHeader (ordertotal inkl. skatt och frakt).


In [ ]:
query_vis4 = """
SELECT
    YEAR(OrderDate) AS OrderYear,
    SUM(TotalDue) AS TotalSales,
    COUNT(DISTINCT SalesOrderID) AS OrderCount
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate)
ORDER BY OrderYear;
"""


df_vis4 = query_df(query_vis4)


df_vis4["Sales_M"] = df_vis4["TotalSales"] / 1_000_000
df_vis4["Orders_k"] = df_vis4["OrderCount"] / 1_000

years = df_vis4["OrderYear"].astype(str)
x = np.arange(len(years))
width = 0.4

fig, ax = plt.subplots(figsize=(10, 6))

bars1 = ax.bar(x - width/2, df_vis4["Sales_M"], width, label="Försäljning (miljoner)", alpha=0.8)
bars2 = ax.bar(x + width/2, df_vis4["Orders_k"], width, label="Antal ordrar (tusental)", alpha=0.8)

ax.set_title("Försäljning och antal ordrar per år", fontsize=14, fontweight="bold")
ax.set_xlabel("År", fontsize=12)
ax.set_ylabel("Försäljning (miljoner) / Ordrar (tusental)", fontsize=12)

ax.set_xticks(x)
ax.set_xticklabels(years)

ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v, p: f"{v:,.1f}"))


for b in bars1:
    ax.text(b.get_x() + b.get_width()/2, b.get_height(), f"{b.get_height():.1f}M", ha="center", va="bottom", fontsize=9)

for b in bars2:
    ax.text(b.get_x() + b.get_width()/2, b.get_height(), f"{b.get_height():.1f}k", ha="center", va="bottom", fontsize=9)

ax.legend(loc="upper left")
plt.tight_layout()
plt.show()



Obs: För att undvika dubbla y-axlar är försäljning visad i miljoner och antal ordrar i tusental.


In [ ]:
top_sales = df_vis4.loc[df_vis4["TotalSales"].idxmax()]
top_orders = df_vis4.loc[df_vis4["OrderCount"].idxmax()]

print("Högst försäljning:", int(top_sales["OrderYear"]), f"({top_sales['TotalSales']:,.2f})")
print("Flest ordrar:", int(top_orders["OrderYear"]), f"({int(top_orders['OrderCount'])})")


### Insikter – Försäljning och antal ordrar per år

- **Året med högst försäljning:** **2024** – **49,020,486.51**  
- **Året med flest ordrar:** **2024** – **14,244** ordrar  

**Utveckling över tid:**  
Både försäljning och antal ordrar ökar tydligt från **2022** till **2024**, vilket indikerar en stark tillväxt (fler genomförda köp och högre totalintäkt).

**2025 som avvikelse:**  
År **2025** är lägre än 2024 (försäljning **22,364,899.77** och **11,699** ordrar). Eftersom datan endast sträcker sig till **2025-06-29** kan 2025 vara ett **ofullständigt år**, vilket sannolikt förklarar nedgången.


## Visualisering 5: Top 10 produkter efter försäljning

**Affärsfråga:** Vilka 10 produkter genererar mest försäljning?

**Tabeller:**  
- `Production.Product`  
- `Sales.SalesOrderDetail`

**Metod:**  
- JOIN produkt → orderrad  
- Summerar försäljning per produkt  
- Försäljning beräknas som SUM(LineTotal) från Sales.SalesOrderDetail (orderradsbelopp – summeras per produkt).
- Filtrerar fram **Top 10** baserat på total försäljning  
- Sorterar från högst till lägst (högsta överst)  
- Visualiserar resultatet som ett **horisontellt stapeldiagram**



In [ ]:
query_vis5 = """
SELECT TOP (10)
    p.Name AS ProductName,
    pc.Name AS CategoryName,
    SUM(sod.LineTotal) AS TotalSales
FROM Sales.SalesOrderDetail sod
INNER JOIN Production.Product p 
    ON sod.ProductID = p.ProductID
LEFT JOIN Production.ProductSubcategory psc 
    ON p.ProductSubcategoryID = psc.ProductSubcategoryID
LEFT JOIN Production.ProductCategory pc 
    ON psc.ProductCategoryID = pc.ProductCategoryID
GROUP BY p.Name, pc.Name
ORDER BY TotalSales DESC;
"""

df_vis5 = query_df(query_vis5)
df_vis5




fig, ax = plt.subplots(figsize=(10,6))
bars = ax.barh(df_vis5["ProductName"], df_vis5["TotalSales"], color="steelblue", alpha=0.8)

ax.invert_yaxis()

ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: f"{x:,.0f}"))

for bar in bars:
    width = bar.get_width()
    ax.text(
        width,
        bar.get_y() + bar.get_height()/2,
        f"{width:,.0f}",
        va="center",
        ha="left"
    )

ax.set_xlabel("Total försäljning (LineTotal)", fontsize=12)
ax.set_ylabel("Produkt", fontsize=12)
ax.set_title("Top 10 produkter efter försäljning", fontsize=14, fontweight="bold")

# lite extra luft så siffror inte hamnar utanför
ax.set_xlim(0, df_vis5["TotalSales"].max() * 1.12)

plt.tight_layout()
plt.show()




top_product = df_vis5.iloc[0]
top_name = top_product["ProductName"]
top_sales = float(top_product["TotalSales"])

dominant_category = df_vis5["CategoryName"].mode()[0]

print("Top 1 produkt:", top_name, f"({top_sales:,.2f})")
print("Dominerande kategori i Top 10:", dominant_category)


### Insikter – Top 10 produkter efter försäljning

- **#1 produkt:** **Mountain-200 Black, 38** – **4,400,592.80** i total försäljning  

**Vilken kategori dominerar top 10?**  
Top 10 domineras helt av kategorin **Bikes** (alla 10 produkter). Det indikerar att cyklar är den största intäktsdrivaren och att topplistan främst består av modeller i **Mountain-** och **Road-serien**.

**Tolkning:**  
Fokus på lager, marknadsföring och kampanjer för dessa bästsäljande cykelmodeller kan ge stor effekt på intäkterna. Samtidigt kan det vara värt att analysera om andra kategorier (t.ex. Accessories/Clothing) har potential att växa genom merförsäljning kopplad till cykelköp.



## Visualisering 6: Försäljning och antal kunder per region

**Affärsfråga:** Hur skiljer sig försäljningen mellan olika regioner, och hur många unika kunder har varje region?

**Tabeller:**  
- `Sales.SalesTerritory`  
- `Sales.SalesOrderHeader`  
- `Sales.Customer`

**Metod:**  
- JOIN orderheader → territory och kopplar kund per order  
- Beräknar **total försäljning** per region samt **antal unika kunder** per region  
- Försäljning beräknas som SUM(TotalDue) från Sales.SalesOrderHeader (ordertotal inkl. skatt och frakt).
- Unika kunder beräknas som COUNT(DISTINCT CustomerID).
- Sorterar regioner från högst till lägst försäljning  
- Visualiserar resultatet som ett **grupperat stapeldiagram** med två staplar per region (försäljning och antal kunder)



In [ ]:
query_vis6 = """
SELECT
    st.Name AS Region,
    SUM(soh.TotalDue) AS TotalSales,
    COUNT(DISTINCT c.CustomerID) AS UniqueCustomers
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.SalesTerritory st
    ON soh.TerritoryID = st.TerritoryID
INNER JOIN Sales.Customer c
    ON soh.CustomerID = c.CustomerID
GROUP BY st.Name
ORDER BY TotalSales DESC;
"""

df_vis6 = query_df(query_vis6)

# Skala för att kunna visa båda på samma axel (utan twinx)
df_vis6["Sales_M"] = df_vis6["TotalSales"] / 1_000_000           # miljoner
df_vis6["Customers_k"] = df_vis6["UniqueCustomers"] / 1_000      # tusental

regions = df_vis6["Region"]
x = np.arange(len(regions))
width = 0.4

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar(x - width/2, df_vis6["Sales_M"], width, label="Försäljning (miljoner)", alpha=0.8)
bars2 = ax.bar(x + width/2, df_vis6["Customers_k"], width, label="Unika kunder (tusental)", alpha=0.8)

ax.set_title("Försäljning och antal kunder per region", fontsize=14, fontweight="bold")
ax.set_xlabel("Region", fontsize=12)
ax.set_ylabel("Försäljning (miljoner) / Kunder (tusental)", fontsize=12)

ax.set_xticks(x)
ax.set_xticklabels(regions, rotation=45, ha="right")

ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v, p: f"{v:,.1f}"))

# Etiketter på staplar (med enhet)
for b in bars1:
    ax.text(b.get_x() + b.get_width()/2, b.get_height(), f"{b.get_height():.1f}M", ha="center", va="bottom", fontsize=9)

for b in bars2:
    ax.text(b.get_x() + b.get_width()/2, b.get_height(), f"{b.get_height():.1f}k", ha="center", va="bottom", fontsize=9)

ax.legend(loc="upper right")
plt.tight_layout()
plt.show()

# Behåll gärna dina prints/beräkningar efter plotten:
df_vis6["SalesPerCustomer"] = df_vis6["TotalSales"] / df_vis6["UniqueCustomers"]

best = df_vis6.iloc[0]
worst = df_vis6.iloc[-1]

print("Starkast (försäljning):", best["Region"], f"Sales={best['TotalSales']:,.2f}", f"Kunder={int(best['UniqueCustomers'])}")
print("Svagast (försäljning):", worst["Region"], f"Sales={worst['TotalSales']:,.2f}", f"Kunder={int(worst['UniqueCustomers'])}")

top_sales_per_customer = df_vis6.loc[df_vis6["SalesPerCustomer"].idxmax()]
print("Högst försäljning per kund:", top_sales_per_customer["Region"], f"({top_sales_per_customer['SalesPerCustomer']:,.2f})")



Obs: För att undvika dubbla y-axlar visas försäljning i miljoner och antal unika kunder i tusental, så att båda måtten kan jämföras i samma diagram.


### Insikter – Försäljning och antal kunder per region

- **Starkast region:** **Southwest** – **27,150,594.59** i total försäljning och **4,565** unika kunder  
- **Svagast region:** **Germany** – **5,479,819.58** i total försäljning  

**Tolkning:**  
Southwest har både högst försäljning och flest kunder, vilket tyder på att försäljningen i regionen drivs av **volym** (många kunder och många affärer).

Samtidigt har **Northeast** högst försäljning per kund (**137,196.66**), vilket tyder på **färre kunder men högt kundvärde** (t.ex. högre ordervärde per kund eller enstaka stora företagskunder). Detta kan innebära att Northeast har en mer “premium”-kundbas eller att regionen innehåller kunder med större inköp.

**Affärsmässig slutsats:**  
- Southwest: fortsätt optimera för volym (tillgänglighet, snabb leverans, kampanjer som driver många köp).  
- Northeast: fokusera på att behålla och utveckla högvärdeskunder (t.ex. företagsavtal, personliga kundkontakter för större kunder och erbjudanden som passar kunder med högre budget).



## Visualisering 7: Genomsnittligt ordervärde per region och kundtyp

**Affärsfråga:** Vilka regioner har högst/lägst genomsnittligt ordervärde, och skiljer det sig mellan individuella kunder och företagskunder?

**Tabeller:**  
- `Sales.SalesTerritory`  
- `Sales.SalesOrderHeader`  
- `Sales.Customer`  
- `Sales.Store`

**Metod:**  
- JOIN orderheader → territory och kopplar kundinformation  
- Delar upp kundtyp i Store (företag) och Individual (privatperson) baserat på om kunden kan kopplas till tabellen Sales.Store (via Customer.StoreID = Store.BusinessEntityID).
- Genomsnittligt ordervärde per region och kundtyp beräknas som SUM(TotalDue) / antal ordrar (TotalDue = ordertotal inkl. skatt och frakt).
- Totalt genomsnitt per region beräknas i pandas: RegionAvgTotal = (TotalSales per region) / (OrderCount per region).  
- Sortering görs i pandas efter RegionAvgTotal (högst → lägst) och används som ordning i diagrammet.
- Visualiserar resultatet som ett **grupperat stapeldiagram** (Store vs Individual per region)  
- *(Tillägg för tydlighet: en linje kan visas för att indikera totalnivån per region, men huvudjämförelsen görs med staplarna.)*




In [ ]:
query_vis7 = """
SELECT
    st.Name AS Region,
    CASE 
        WHEN s.BusinessEntityID IS NOT NULL THEN 'Store'
        ELSE 'Individual'
    END AS CustomerType,
    SUM(soh.TotalDue) / COUNT(DISTINCT soh.SalesOrderID) AS AvgOrderValue,
    SUM(soh.TotalDue) AS TotalSales,
    COUNT(DISTINCT soh.SalesOrderID) AS OrderCount
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.SalesTerritory st
    ON soh.TerritoryID = st.TerritoryID
INNER JOIN Sales.Customer c
    ON soh.CustomerID = c.CustomerID
LEFT JOIN Sales.Store s
    ON c.StoreID = s.BusinessEntityID
GROUP BY
    st.Name,
    CASE 
        WHEN s.BusinessEntityID IS NOT NULL THEN 'Store'
        ELSE 'Individual'
    END;
"""

df_vis7 = query_df(query_vis7)
df_vis7






region_totals = df_vis7.groupby("Region")[["TotalSales", "OrderCount"]].sum()
region_totals["RegionAvgTotal"] = region_totals["TotalSales"] / region_totals["OrderCount"]


df_pivot = df_vis7.pivot_table(
    index="Region",
    columns="CustomerType",
    values="AvgOrderValue",
    aggfunc="first"
)



df_pivot = df_pivot.reindex(columns=["Store", "Individual"])


df_plot = df_pivot.join(region_totals["RegionAvgTotal"]).sort_values("RegionAvgTotal", ascending=False)

regions = df_plot.index
x = np.arange(len(regions))
width = 0.4

fig, ax = plt.subplots(figsize=(12,6))
ax.bar(x - width/2, df_plot["Store"], width, label="Store", color="steelblue", alpha=0.8)
ax.bar(x + width/2, df_plot["Individual"], width, label="Individual", color="orange", alpha=0.8)



ax.set_title("Genomsnittligt ordervärde per region och kundtyp", fontsize=14, fontweight="bold")
ax.set_xlabel("Region", fontsize=12)
ax.set_ylabel("Genomsnittligt ordervärde (TotalDue per order)", fontsize=12)

ax.set_xticks(x)
ax.set_xticklabels(regions, rotation=45, ha="right")

ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v, p: f"{v:,.0f}"))
ax.legend()

plt.tight_layout()
plt.show()





type_totals = df_vis7.groupby("CustomerType")[["TotalSales", "OrderCount"]].sum()
type_totals["AvgOverall"] = type_totals["TotalSales"] / type_totals["OrderCount"]

best_region = region_totals["RegionAvgTotal"].idxmax()
worst_region = region_totals["RegionAvgTotal"].idxmin()

top_combo = df_vis7.loc[df_vis7["AvgOrderValue"].idxmax()]

print("Store totalt:", type_totals.loc["Store","AvgOverall"])
print("Individual totalt:", type_totals.loc["Individual","AvgOverall"])
print("Högst region (totalt):", best_region, region_totals.loc[best_region,"RegionAvgTotal"])
print("Lägst region (totalt):", worst_region, region_totals.loc[worst_region,"RegionAvgTotal"])
print("Högst kombination:", top_combo["Region"], top_combo["CustomerType"], top_combo["AvgOrderValue"])



Obs: Staplarna visar genomsnittligt ordervärde separat för Store (företag) och Individual (privatperson) i varje region. Kundtyp definieras utifrån om kunden kan kopplas till Sales.Store. Regionerna är sorterade efter total genomsnittlig orderstorlek (alla ordrar i regionen, oavsett kundtyp).


### Insikter – Genomsnittligt ordervärde per region och kundtyp

**Store vs Individual:**  
I denna analys definieras kundtyp så här:  
- **Store** = kund som har en koppling till tabellen `Sales.Store` (företagskund)  
- **Individual** = kund utan sådan koppling (privatperson)

Resultatet visar att **Store-kunder** har ett tydligt högre genomsnittligt ordervärde än **Individual** (totalt ca **23,850.62** jämfört med **1,172.90**). Mönstret syns i alla regioner, vilket tyder på att företagskunder generellt lägger större ordrar.

**Högst/lägst region (totalt):**  
- **Högst:** **Central** – **23,151.43**  
- **Lägst:** **Australia** – **1,726.49**

**Högsta region + kundtyp-kombination:**  
- **Southwest + Store:** **27,739.06** i genomsnittligt ordervärde

**Tolkning:**  
Skillnaderna drivs sannolikt av **kundmix** (andel Store vs Individual) och att företagskunder ofta gör större inköp per order. Regioner med fler företagskunder eller större företagsordrar får därför ett högre genomsnittligt ordervärde.


# Djupanalys: Alternativ A – Regional försäljningsoptimering

## Syfte
Målet är att analysera regionala skillnader i försäljning för att förstå:
- vilken region som presterar bäst/sämst
- vilka produktkategorier som driver försäljning i olika regioner
- om det finns säsongsmönster per region
- vilka konkreta förbättringar/rekommendationer som kan föreslås

## Metod
1. **Regional översikt (KPI:er)**  
   Hämta total försäljning, antal ordrar, antal kunder och genomsnittligt ordervärde per region för att identifiera starkaste och svagaste regionerna.

2. **Produktmix per region**  
   Beräkna försäljning per region och produktkategori. Skapa en pivot-tabell (Region × Kategori) i pandas och visualisera med heatmap.

3. **Säsongsmönster per region**  
   Beräkna försäljning per region och månad. Skapa pivot-tabell (Månad × Region) och visualisera med linjediagram. Jämför regionernas trender.

4. **Rekommendationer**  
   Föreslå konkreta åtgärder baserat på resultaten (t.ex. satsning på specifika kategorier i specifika regioner, säsongskampanjer, cross-sell).



## Alternativ A1: Regional översikt (KPI per region)

**Affärsfråga:** Vilken region presterar bäst/sämst, och beror det på volym (många kunder/ordrar) eller högt värde per kund?

**Tabeller:**  
- `Sales.SalesTerritory`  
- `Sales.SalesOrderHeader`  
- `Sales.Customer`

**Metod:**  
- Hämtar KPI: total försäljning, antal ordrar, antal kunder, genomsnittligt ordervärde samt försäljning per kund  
- Sorterar efter total försäljning för att se starkast/svagast  
- Visualiserar för att jämföra **volym** (många kunder/ordrar) vs **kundvärde** (hög försäljning per kund)



In [ ]:
query_alt_a_1 = """
SELECT
    st.Name AS Region,
    SUM(soh.TotalDue) AS TotalSales,
    COUNT(DISTINCT soh.SalesOrderID) AS OrderCount,
    COUNT(DISTINCT soh.CustomerID) AS UniqueCustomers,
    SUM(soh.TotalDue) / COUNT(DISTINCT soh.SalesOrderID) AS AvgOrderValue,
    SUM(soh.TotalDue) / COUNT(DISTINCT soh.CustomerID) AS SalesPerCustomer
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.SalesTerritory st
    ON soh.TerritoryID = st.TerritoryID
GROUP BY st.Name
ORDER BY TotalSales DESC;
"""
df_alt_a_1 = query_df(query_alt_a_1)
df_alt_a_1




import numpy as np

regions = df_alt_a_1["Region"]
sales = df_alt_a_1["TotalSales"]
sales_per_customer = df_alt_a_1["SalesPerCustomer"]

x = np.arange(len(regions))
width = 0.4

fig, ax1 = plt.subplots(figsize=(12,6))
ax2 = ax1.twinx()

bars1 = ax1.bar(x - width/2, sales, width, label="Total försäljning", color="steelblue", alpha=0.8)
bars2 = ax2.bar(x + width/2, sales_per_customer, width, label="Försäljning per kund", color="orange", alpha=0.8)

ax1.set_title("Alternativ A: KPI per region", fontsize=14, fontweight="bold")
ax1.set_xlabel("Region", fontsize=12)
ax1.set_ylabel("Total försäljning (TotalDue)", fontsize=12)
ax2.set_ylabel("Försäljning per kund (TotalDue)", fontsize=12)

ax1.set_xticks(x)
ax1.set_xticklabels(regions, rotation=45, ha="right")

ax1.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v, p: f"{v:,.0f}"))
ax2.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v, p: f"{v:,.0f}"))

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1.legend(
    h1 + h2, l1 + l2,
    loc="upper right",
    bbox_to_anchor=(0.80, 1.0),
    frameon=True
)

plt.tight_layout()
plt.show()


Obs: Diagrammet använder två y-axlar eftersom total försäljning och försäljning per kund har olika skala.


### Insikter – Alternativ A1

- **Starkast region (total försäljning):** Southwest – **27,150,594.59**  
  Southwest drivs främst av volym: flest kunder (**4,565**) och många ordrar (**6,224**).

- **Svagast region (total försäljning):** Germany – **5,479,819.58**

- **Högst försäljning per kund:**  
  - Northeast – **137,196.66**  
  - Central – **129,178.25**  

Trots få kunder har Northeast och Central mycket högt värde per kund. Det tyder på färre men större affärer (t.ex. större kunder eller större orderstorlekar) och kan kräva en annan strategi än i volymregioner som Southwest, exempelvis mer fokus på att behålla och utveckla större kunder.


## Alternativ A2: Produktmix per region (Region × Kategori)

**Affärsfråga:** Vilka produktkategorier säljer bäst i olika regioner?

**Tabeller:**  
- `Sales.SalesTerritory`  
- `Sales.SalesOrderHeader`  
- `Sales.SalesOrderDetail`  
- `Production.Product`  
- `Production.ProductSubcategory`  
- `Production.ProductCategory`

**Metod:**  
- Hämtar försäljning per region och kategori  
- Skapar pivot-tabell i pandas (Region × Kategori) för att jämföra regioner på ett tydligt sätt  
- Visualiserar pivot-tabellen med heatmap för att se mönster i produktmix



In [ ]:
query_alt_a_2 = """
SELECT
    st.Name AS Region,
    pc.Name AS CategoryName,
    SUM(sod.LineTotal) AS TotalSales
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.SalesTerritory st
    ON soh.TerritoryID = st.TerritoryID
INNER JOIN Sales.SalesOrderDetail sod
    ON soh.SalesOrderID = sod.SalesOrderID
INNER JOIN Production.Product p
    ON sod.ProductID = p.ProductID
INNER JOIN Production.ProductSubcategory psc
    ON p.ProductSubcategoryID = psc.ProductSubcategoryID
INNER JOIN Production.ProductCategory pc
    ON psc.ProductCategoryID = pc.ProductCategoryID
GROUP BY st.Name, pc.Name
ORDER BY st.Name, pc.Name;
"""

df_alt_a_2 = query_df(query_alt_a_2)
df_alt_a_2


pivot_alt_a_2 = df_alt_a_2.pivot_table(
    index="Region",
    columns="CategoryName",
    values="TotalSales",
    aggfunc="sum",
    fill_value=0
)


pivot_alt_a_2 = pivot_alt_a_2.loc[pivot_alt_a_2.sum(axis=1).sort_values(ascending=False).index]
pivot_alt_a_2 = pivot_alt_a_2[pivot_alt_a_2.sum(axis=0).sort_values(ascending=False).index]

heat_vals = np.log1p(pivot_alt_a_2.values)

fig, ax = plt.subplots(figsize=(10,6))
im = ax.imshow(heat_vals, aspect="auto")

ax.set_title("Heatmap: Försäljning per region och kategori (log-skala)", fontsize=14, fontweight="bold")
ax.set_xlabel("Kategori")
ax.set_ylabel("Region")

ax.set_xticks(np.arange(pivot_alt_a_2.shape[1]))
ax.set_xticklabels(pivot_alt_a_2.columns, rotation=45, ha="right")

ax.set_yticks(np.arange(pivot_alt_a_2.shape[0]))
ax.set_yticklabels(pivot_alt_a_2.index)

plt.colorbar(im, ax=ax, label="log(1 + försäljning (LineTotal))")
plt.tight_layout()
plt.show()



pivot_alt_a_2




Obs: Heatmapen använder log-skala (log(1 + försäljning)) för att göra skillnader i mindre kategorier synliga när Bikes dominerar i absoluta tal.

### Insikter – Alternativ A2

Heatmapen visar att **Bikes** dominerar försäljningen i samtliga regioner. Exempelvis står Bikes för största delen av försäljningen i:
- Southwest – **20,803,673.94**
- Northwest – **13,882,022.98**
- Canada – **13,457,682.98**

Utöver Bikes är **Components** tydligt näst starkast i flera regioner, särskilt i:
- Southwest – **2,692,201.20**
- Canada – **2,244,470.02**

Det tyder på potential för merförsäljning (t.ex. delar/komponenter i samband med cykelköp) i regioner med stark cykelförsäljning.


## Alternativ A3: Säsongsmönster per region

**Affärsfråga:** Finns säsongsmönster per region, och skiljer de sig mellan regioner?

**Tabeller:**  
- `Sales.SalesTerritory`  
- `Sales.SalesOrderHeader`

**Metod:**  
- Hämtar total försäljning per region och månad  
- Skapar pivot-tabell i pandas (Månad × Region) för att kunna jämföra över tid  
- Väljer de regioner med högst total försäljning för att undvika att diagrammet blir för rörigt  
- Visualiserar med linjediagram över tid



In [ ]:
query_alt_a_3 = """
SELECT
    st.Name AS Region,
    DATEFROMPARTS(YEAR(soh.OrderDate), MONTH(soh.OrderDate), 1) AS MonthStart,
    SUM(soh.TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader soh
INNER JOIN Sales.SalesTerritory st
    ON soh.TerritoryID = st.TerritoryID
GROUP BY st.Name, DATEFROMPARTS(YEAR(soh.OrderDate), MONTH(soh.OrderDate), 1)
ORDER BY MonthStart, st.Name;
"""

df_alt_a_3 = query_df(query_alt_a_3)
df_alt_a_3["MonthStart"] = pd.to_datetime(df_alt_a_3["MonthStart"])
df_alt_a_3




pivot_alt_a_3 = df_alt_a_3.pivot_table(
    index="MonthStart",
    columns="Region",
    values="TotalSales",
    aggfunc="sum",
    fill_value=0
).sort_index()

top3_regions = pivot_alt_a_3.sum(axis=0).sort_values(ascending=False).head(3).index
df_top3 = pivot_alt_a_3[top3_regions]

fig, ax = plt.subplots(figsize=(12,6))
for r in top3_regions:
    ax.plot(df_top3.index, df_top3[r], label=r)

ax.set_title("Säsongsmönster: Försäljning per månad (Top 3 regioner)", fontsize=14, fontweight="bold")
ax.set_xlabel("Månad", fontsize=12)
ax.set_ylabel("Total försäljning (TotalDue)", fontsize=12)
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


top3 = list(top3_regions)  # Southwest, Canada, Northwest

for r in top3:
    s = df_top3[r]
    print(f"\n{r}")
    print("Högsta månad:", s.idxmax().strftime("%Y-%m"), f"({s.max():,.2f})")
    print("Lägsta månad:", s.idxmin().strftime("%Y-%m"), f"({s.min():,.2f})")




### Insikter – Alternativ A3

**Top 3 regioner (baserat på total försäljning):** Southwest, Canada och Northwest.

Regionerna följer en liknande trend över tid där Southwest generellt ligger högst, medan Canada och Northwest ligger på en lägre nivå men ofta rör sig i samma riktning. Detta tyder på att regionerna påverkas av liknande säsongsvariationer.

- **Southwest:** högst **2023-07** (**1,265,968.34**), lägst **2025-06** (**9,432.32**)  
- **Canada:** högst **2024-10** (**906,430.48**), lägst **2025-06** (**11,123.11**)  
- **Northwest:** högst **2024-06** (**930,885.46**), lägst **2025-06** (**10,441.15**)  

Att alla tre regioner har sin lägsta månad i **2025-06** stämmer överens med Visualisering 3 där juni 2025 identifierades som en tydlig avvikelse med ovanligt låg försäljning och förändrad produktmix. Det tyder på en gemensam faktor som påverkar flera regioner samtidigt (t.ex. att högintäktsprodukter som Bikes saknas den månaden).




## Alternativ A4: Top 3 kategorier per region

**Affärsfråga:** Vilka produktkategorier driver försäljningen i varje region, och skiljer det sig mellan regioner?

**Tabeller:**  
- `Sales.SalesTerritory`  
- `Sales.SalesOrderHeader`  
- `Sales.SalesOrderDetail`  
- `Production.Product`  
- `Production.ProductSubcategory`  
- `Production.ProductCategory`

**Metod:**  
- Hämtar försäljning per region och produktkategori  
- Rangordnar kategorier per region efter total försäljning (Top 3)  
- Jämför **starkaste regionen (Southwest)** med **svagaste regionen (Germany)** för att se skillnader i produktmix  
- Visualiserar resultatet som ett stapeldiagram för en tydlig jämförelse




In [ ]:
query_alt_a_4 = """
WITH rc AS (
    SELECT
        st.Name AS Region,
        pc.Name AS CategoryName,
        SUM(sod.LineTotal) AS TotalSales
    FROM Sales.SalesOrderHeader soh
    INNER JOIN Sales.SalesTerritory st
        ON soh.TerritoryID = st.TerritoryID
    INNER JOIN Sales.SalesOrderDetail sod
        ON soh.SalesOrderID = sod.SalesOrderID
    INNER JOIN Production.Product p
        ON sod.ProductID = p.ProductID
    INNER JOIN Production.ProductSubcategory psc
        ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    INNER JOIN Production.ProductCategory pc
        ON psc.ProductCategoryID = pc.ProductCategoryID
    GROUP BY st.Name, pc.Name
)
SELECT *
FROM (
    SELECT
        Region,
        CategoryName,
        TotalSales,
        DENSE_RANK() OVER (PARTITION BY Region ORDER BY TotalSales DESC) AS CategoryRank
    FROM rc
) x
WHERE CategoryRank <= 3
ORDER BY Region, CategoryRank;
"""
df_alt_a_4 = query_df(query_alt_a_4)
df_alt_a_4




best_region = df_alt_a_1.iloc[0]["Region"]
worst_region = df_alt_a_1.iloc[-1]["Region"]

df_bw = df_alt_a_4[df_alt_a_4["Region"].isin([best_region, worst_region])].copy()


pivot_bw = df_bw.pivot_table(
    index="CategoryName",
    columns="Region",
    values="TotalSales",
    aggfunc="sum",
    fill_value=0
)

pivot_bw = pivot_bw.sort_values(by=best_region, ascending=False)

categories = pivot_bw.index
x = np.arange(len(categories))
width = 0.4

fig, ax = plt.subplots(figsize=(10,6))
ax.bar(x - width/2, pivot_bw[best_region], width, label=best_region, alpha=0.8)
ax.bar(x + width/2, pivot_bw[worst_region], width, label=worst_region, alpha=0.8)

ax.set_title("Alternativ A: Top-kategorier i starkaste vs svagaste region", fontsize=14, fontweight="bold")
ax.set_xlabel("Kategori")
ax.set_ylabel("Total försäljning (LineTotal)")

ax.set_xticks(x)
ax.set_xticklabels(categories, rotation=45, ha="right")
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v, p: f"{v:,.0f}"))

ax.legend()
plt.tight_layout()
plt.show()



### Insikter – Alternativ A4

**Southwest (starkast region):**
- Bikes – **20,803,673.94**
- Components – **2,692,201.20**
- Clothing – **434,914.62**

**Germany (svagast region):**
- Bikes – **4,382,176.51**
- Components – **337,786.52**
- Clothing – **98,113.92**

**Tolkning:**  
Båda regioner har samma top-3 kategorier (Bikes, Components och Clothing), vilket tyder på att produktmixen i grunden är liknande. Skillnaden ligger främst i **volym**: Southwest har betydligt högre försäljning i samtliga tre kategorier, särskilt i Bikes. Det stärker bilden av att Southwest är en volymregion med många affärer, medan Germany säljer samma typer av produkter men i mindre omfattning.

**Andel av regionens totala försäljning (procent):**
- **Southwest:** Bikes **86.02%**, Components **11.13%**, Clothing **1.80%** (top 3-kategorierna = **98.95%**)  
- **Germany:** Bikes **89.15%**, Components **6.87%**, Clothing **2.00%** (top 3-kategorierna = **98.02%**)  

Det visar att båda regionerna är starkt beroende av Bikes, och att beroendet är ännu tydligare i Germany (högre Bikes-andel). Southwest har något större andel Components, vilket kan tyda på bättre merförsäljning av delar/komponenter kopplat till cykelförsäljningen.



## Rekommendationer (Alternativ A)

- **Southwest** har högst total försäljning och många kunder/ordrar. Fortsätt prioritera **Bikes** och öka merförsäljningen av **Components** (t.ex. rekommenderade delar/uppgraderingar i samband med cykelköp).

- **Central** och **Northeast** har mycket hög försäljning per kund men få kunder. Här kan små ökningar i kundbasen ge stor effekt. Satsa på att hitta fler kunder med liknande beteende och arbeta mer aktivt med att behålla och utveckla större kunder.

- Eftersom **Bikes dominerar i alla regioner** bör lager och kampanjer planeras med cykelförsäljningen som utgångspunkt. **Components** är en tydlig kompletterande kategori som kan paketeras med Bikes.

- Säsongsmönster i top-regionerna kan användas för att planera lager och marknadsföring. **Juni 2025** är en tydlig avvikelse (Visualisering 3) och bör därför tolkas med försiktighet.




## Reflektion

### Varför valde jag Alternativ A?
Jag valde **Alternativ A (Regional försäljningsoptimering)** eftersom jag redan hade analyserat regioner i Visualisering 6 och 7 och ville bygga vidare med en djupare analys av **produktmix** och **säsongsmönster**. Målet var att förstå varför vissa regioner presterar bättre än andra och ge konkreta förbättringsförslag.

### Varför aggregering i SQL vs pandas?
Jag gjorde den grundläggande aggregeringen i **SQL** (SUM/COUNT/GROUP BY) eftersom databasen är effektiv på att sammanställa stora datamängder och jag kan hämta tillbaka exakt den nivå av data jag behöver.  
I **pandas** använde jag pivot-tabeller och sortering för att snabbt kunna ändra perspektiv (t.ex. Region × Kategori eller Månad × Region) och skapa visualiseringar.

### Varför dessa diagram?
- **Staplar (KPI per region):** tydligt för att jämföra regioner och skilja på volym vs kundvärde  
- **Heatmap (Region × Kategori):** visar snabbt mönster i produktmix mellan regioner  
- **Linjediagram (månadsförsäljning per region):** bra för trender och säsongsmönster över tid  
- **Grupperade staplar (starkast vs svagast region):** gör skillnader i produktmix lätt att jämföra direkt

### Hur hjälpte pivot-tabeller mig?
Pivot-tabeller gjorde det enkelt att:
- jämföra **Region × Kategori** i heatmapen  
- jämföra **Månad × Region** över tid  
Det gav en tydlig överblick och gjorde det lättare att se mönster som inte syns i rå data.

### Utmaningar och hur jag löste dem
En utmaning var att vissa mått har väldigt olika skala (t.ex. total försäljning vs försäljning per kund). Jag löste det genom att använda tydliga visualiseringar och komplettera med siffror och förklarande text så att resultaten inte misstolkas.




## Sammanfattning

Analysen visar att försäljningen i AdventureWorks till stor del drivs av **Bikes**, både på kategorinivå och bland de mest säljande produkterna. Försäljningen varierar över tid och uppvisar ett tydligt säsongsmönster där vissa månader återkommer som starkare än andra.

Regionalt är **Southwest** starkast i total försäljning och drivs främst av volym (många kunder och ordrar), medan **Germany** är svagast. Samtidigt sticker **Central** och **Northeast** ut med väldigt högt värde per kund, vilket tyder på färre men större kunder/order i dessa regioner.

### Rekommendationer
- Fortsätt prioritera **Bikes** som huvudkategori och stärk merförsäljning till **Components** i regioner med hög cykelförsäljning.
- För regioner som **Central** och **Northeast**: fokusera på att behålla och utveckla större kunder samt bredda kundbasen, eftersom få kunder står för stor försäljning.
- Planera lager och kampanjer utifrån säsongsmönster. **Juni 2025** är en tydlig avvikelse (Visualisering 3) och bör tolkas med försiktighet.
